In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris

In [ ]:
data = load_iris()
X = tf.constant(data['data'])
dataset = tf.data.Dataset.from_tensors(X)

In [ ]:
# dataset transformations - apply standard scaling

In [ ]:
batches = dataset.batch(50)
mean = batches.map(tf.reduce_mean)
iterator = mean.make_one_shot_iterator()
elem = iterator.get_next()

with tf.Session() as sess:
    print(sess.run(elem))

In [ ]:
np.mean(data['data'])

In [ ]:
# Columnwise mean?

In [ ]:
from functools import partial
colMean = partial(tf.reduce_mean, axis=0)

In [ ]:
dataset = tf.data.Dataset.from_tensors(X)
averaged = dataset.map(colMean)
it = averaged.make_one_shot_iterator()
elem = it.get_next()

with tf.Session() as sess:
    print(sess.run(elem))

In [ ]:
# What went wrong?

In [ ]:
colMean = partial(tf.reduce_mean, axis=1)
dataset = tf.data.Dataset.from_tensors(tf.transpose(X))
averaged = dataset.map(colMean)
it = averaged.make_one_shot_iterator()
elem = it.get_next()

with tf.Session() as sess:
    print(sess.run(elem))

In [ ]:
np.mean(data['data'], axis=0)

In [ ]:
# variance?

In [ ]:
dataset = tf.data.Dataset.from_tensors(tf.transpose(X))
variance = dataset.map(lambda x: tf.nn.moments(x, axes=[1])[1])
it = variance.make_one_shot_iterator()
elem = it.get_next()

with tf.Session() as sess:
    print(sess.run(elem))

In [ ]:
np.var(data['data'], axis=0)

In [ ]:
# Putting it all together

In [ ]:
colMean = partial(tf.reduce_mean, axis=1)
colStd = lambda x: tf.sqrt(tf.nn.moments(x, axes=[1])[1])

dataset = tf.data.Dataset.from_tensors(tf.transpose(X))
mean = dataset.map(colMean).make_one_shot_iterator().get_next()
std = dataset.map(colStd).make_one_shot_iterator().get_next()

centered = dataset.map(lambda x: x - tf.reshape(mean, (4, 1)))
# scaled = centered.map(lambda x: x / tf.reshape(std, (4, 1)))

it = centered.make_initializable_iterator()
elem = it.get_next()

with tf.Session() as sess:
    sess.run(it.initializer, feed_dict={'max_value': 5})
    print(sess.run(elem))